In [ ]:
%pylab inline
%matplotlib inline

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import sklearn
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import datetime
import pickle

### training

In [ ]:
# train_df = pd.read_csv('./train.data.new.sum.csv')
train_df = pd.read_csv('./train.data.new_prod.csv')

In [ ]:
train_c7 = pd.read_csv('./c7.train.csv')
# train_c7.c7

In [ ]:
train_df.Feature_7 = train_c7.c7_50
train_df = pd.concat([train_df.Id, train_c7.c7, train_df.drop('Id', axis=1)], axis=1)
train_df.Feature_7.head()

In [ ]:
train_df.c7.head()

In [ ]:
feature_list = train_df.columns[1:-64]
feat_reg = r'Feature_|MOM|ALMA|OSMA|MACD|RSI|f7_c100|cumsum'
# feature_list = train_df.filter(regex=feat_reg, axis=1).columns#[1:-62]
# feature_list = train_df.filter(regex=r'sd|mean|Feature_7|MOM|MACD|OSMA', axis=1).columns #~0.61
# feature_list = train_df.filter(regex=r'sd|mean|Feature|ALMA|EMA', axis=1).columns#[0:-62]
# X = train_df.ix[:, indexes[0:476]]
# X = train_df.ix[:, feature_list]

In [ ]:
# X = pd.concat([pd.get_dummies(train_df['Feature_7']).rename(columns=lambda x: 'Feature_7_' + str(x)),
#                            X], axis=1)
# X.drop('Feature_7', axis=1, inplace=True)
X = train_df.ix[:, feature_list]
X.drop('Feature_7', axis=1, inplace=True)

In [ ]:
X.head()
# X.Feature_7.head()

In [ ]:
# input parametrs of training
param_train_size = 0.7
cv_param = {'max_depth': [16, 12],
            'n_estimators': [250, 150],
            'learning_rate': [0.15]}
# suffix_file = "_clf_all_features.pkl"
suffix_file = "_clf_" + feat_reg.replace('|', '_') + ".pkl"
y_list = ['Ret_PlusOne', 'Ret_PlusTwo']

In [ ]:
median_neg = dict()
median_pos = dict()
X_trains = dict()
X_tests = dict()
y_trains = dict()
y_tests = dict()
X_valids = dict()
y_valids = dict()
clfs = dict()
fitteds = dict()
cut_dict = dict()
xgb_models = dict()
matrix_trains = dict()
matrix_tests = dict()

# REGRESSION


In [ ]:
X = pd.concat([X, train_df.Weight_Daily], axis=1)

In [ ]:
max_ind = np.argmax(X.ix[:, -120:-1].as_matrix(), axis=1)
min_ind = np.argmin(X.ix[:, -120:-1].as_matrix(), axis=1)

X = pd.concat([pd.DataFrame(min_ind, columns=['min_ret']),
               pd.DataFrame(max_ind, columns=['max_ret']),
               X], axis=1)

In [ ]:
feature_list = X.filter(regex=r'cumprod', axis=1).columns

max_ind = np.argmax(X.ix[:, feature_list].as_matrix(), axis=1)
min_ind = np.argmin(X.ix[:, feature_list].as_matrix(), axis=1)

X = pd.concat([pd.DataFrame(min_ind, columns=['min_cum']),
               pd.DataFrame(max_ind, columns=['max_cum']), X], axis=1)

In [ ]:
feature_list = X.filter(regex=r'cumprod', axis=1).columns

std =    np.std(X.ix[:, -120:-1].as_matrix(), axis=1)
median = np.median(X.ix[:, -120:-1].as_matrix(), axis=1)

X = pd.concat([pd.DataFrame(std, columns=['std']),
               pd.DataFrame(median, columns=['median']), X], axis=1)

In [ ]:
# X.drop('Weight_Daily', axis=1, inplace=True)
# X = pd.concat([X, train_df.Weight_Intraday], axis=1)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
# X1 = StandardScaler().fit_transform(X.ix[:, :-121])
# X = pd.DataFrame(X1, columns=X.columns[:-1])
# X = pd.concat([X, train_df.Weight_Daily], axis=1)
# Y  = train_df.filter(regex="MOM|MACD|cums", axis=1)
new_columns = [x + '_Scaler' for x in X.ix[:, :-1].columns.tolist()]
scal = StandardScaler()
X1 = scal.fit_transform(X.ix[:, :-1])
X = pd.concat([pd.DataFrame(X1, columns=new_columns), 
              X], axis=1)
# X.max_cum

In [ ]:
# X1 = X.copy()
X = X1.copy()
X = X.ix[:, l[0:950]]
# y_tests[y_label]
X.head()
# X.Feature_7
# len(v)

In [ ]:
import gc
gc.collect()

In [ ]:
X_Dtrains = dict()
X_Dvalids = dict()
X_Dtests = dict()
X_Dtrains_CL = dict()
X_Dvalids_CL = dict()
X_Dtests_CL = dict()
X_Dtrains_CL_2 = dict()
X_Dvalids_CL_2 = dict()
X_Dtests_CL_2 = dict()
xgb_models_cl = dict()
xgb_models_cl_2 = dict()

In [ ]:
y_list = ['Ret_PlusOne', 'Ret_PlusTwo']
# y_list = ['Ret_121', 'Ret_170']
for y_label in y_list:
#     y_label = 'Ret_PlusOne'
    median_neg[y_label] = np.median(train_df[y_label][train_df[y_label] < 0])
    median_pos[y_label] = np.median(train_df[y_label][train_df[y_label] > 0])

    cut_dict[y_label] = train_df[y_label]

    X_trains[y_label], X_tests[y_label], y_trains[y_label], y_tests[y_label] = train_test_split(
        X, cut_dict[y_label], train_size=param_train_size, random_state = 3124356)

    
    llen = X_tests[y_label].shape[0]/2
    X_valids[y_label] = X_tests[y_label][0:llen]
    X_tests[y_label] = X_tests[y_label][llen:]
    y_valids[y_label] = y_tests[y_label][0:llen]
    y_tests[y_label] = y_tests[y_label][llen:]
#     X_Dtrains[y_label] = xgb.DMatrix(X_trains[y_label].ix[:, :-1],
#                                        label=y_trains[y_label],
#                                        weight=X_trains[y_label].ix[:, -1])
    
#     X_Dvalids[y_label] = xgb.DMatrix(X_valids[y_label].ix[:, :-1],
#                                        label=y_valids[y_label],
#                                        weight=X_valids[y_label].ix[:, -1])
    
#     X_Dtests[y_label] = xgb.DMatrix(X_tests[y_label].ix[:, :-1],
#                                        label=y_tests[y_label],
#                                        weight=X_tests[y_label].ix[:, -1])
    
    
    X_Dtrains_CL[y_label] = xgb.DMatrix(X_trains[y_label].ix[:, :-1],
                                       label=pd.cut(y_trains[y_label],
                                           (-1, median_neg[y_label] * .7 , median_pos[y_label] * .7, 1),
                                           labels=(0, 1, 2)),
                                       weight=X_trains[y_label].ix[:, -1])
    
    X_Dvalids_CL[y_label] = xgb.DMatrix(X_valids[y_label].ix[:, :-1],
                                       label=pd.cut(y_valids[y_label],
                                           (-1, median_neg[y_label] * .7, median_pos[y_label] * .7, 1),
                                           labels=(0, 1, 2)),
                                       weight=X_valids[y_label].ix[:, -1])
    
    X_Dtests_CL[y_label] = xgb.DMatrix(X_tests[y_label].ix[:, :-1],
                                       label=pd.cut(y_tests[y_label],
                                           (-1, median_neg[y_label] * .7, median_pos[y_label] * .7, 1),
                                           labels=(0, 1, 2)),
                                       weight=X_tests[y_label].ix[:, -1])
    
    X_Dtrains_CL_2[y_label] = xgb.DMatrix(X_trains[y_label].ix[:, :-1],
                                       label=pd.cut(y_trains[y_label],
                                           (-1, 0, 1),
                                           labels=(0, 1)),
                                       weight=X_trains[y_label].ix[:, -1])
    
    X_Dvalids_CL_2[y_label] = xgb.DMatrix(X_valids[y_label].ix[:, :-1],
                                       label=pd.cut(y_valids[y_label],
                                           (-1, 0, 1),
                                           labels=(0, 1)),
                                       weight=X_valids[y_label].ix[:, -1])
    
    X_Dtests_CL_2[y_label] = xgb.DMatrix(X_tests[y_label].ix[:, :-1],
                                       label=pd.cut(y_tests[y_label],
                                           (-1, 0, 1),
                                           labels=(0, 1)),
                                       weight=X_tests[y_label].ix[:, -1])
    
    break

In [ ]:
def wmae(preds, dtrain):
    target = dtrain.get_label()
    weight = dtrain.get_weight()
    x1 = np.abs(target - preds)
    x2 = x1 * weight
    x3 = np.mean(x2)
    # Yes, this is needlessly complex, but I had debugging issues
    return 'wmae', x3


for y_label in y_list:

#     watchlist = [(X_Dtrains[y_label], 'train'),
#                  (X_Dvalids[y_label], 'valid'), 
#                  (X_Dtests[y_label], 'test')]
    
    param = {'max_depth':11, 'eta':0.04, 'silent':5}
    param_CL = {'max_depth':11, 'eta':0.04,
             'silent':5, 'eval_metric':'merror', 'objective':'multi:softprob',
            'num_class':3}
    num_round = 221
    
#     print 'valid', wmae(0, X_Dvalids[y_label]), 'test', wmae(0, X_Dtests[y_label]) 
    print(y_label)
    print(str(datetime.datetime.now()))
    sys.stdout.flush()
# #     regression
                                       
#     xgb_models[y_label] = xgb.train(param, X_Dtrains[y_label],
#                                     num_boost_round=num_round,
#                                     evals=[(X_Dtrains[y_label], 'train'),
#                                      (X_Dvalids[y_label], 'valid'), 
#                                      (X_Dtests[y_label], 'test')],
#                                     feval=wmae,
#                                     early_stopping_rounds = 10)



    xgb_models_cl[y_label] = xgb.train({'max_depth':16, 'eta':0.1,
                             'silent':5, 'eval_metric':'merror', 'objective':'multi:softprob',
                            'num_class':3},
                                       X_Dtrains_CL[y_label],
                                    num_boost_round=num_round,
                                    evals=[(X_Dtrains_CL[y_label], 'train'),
                                     (X_Dvalids_CL[y_label], 'valid'), 
                                     (X_Dtests_CL[y_label], 'test')],
                                   early_stopping_rounds = 10)
    
    xgb_models_cl_2[y_label] = xgb.train({'max_depth':11, 'eta':0.04,
             'silent':5, 'eval_metric':'merror', 'objective':'multi:softprob', 'num_class':2},
             X_Dtrains_CL_2[y_label],
             num_boost_round=num_round, 
             evals=[(X_Dtrains_CL_2[y_label], 'train'),
              (X_Dvalids_CL_2[y_label], 'valid'), 
              (X_Dtests_CL_2[y_label], 'test')],
               early_stopping_rounds = 10)
    print(str(datetime.datetime.now()))
    sys.stdout.flush()
    break

In [ ]:
pickle.dump(xgb_models[y_label], open('all_features_and_scales_all_featuresreg', "wb"))
pickle.dump(xgb_models_cl_2[y_label], open('all_features_and_scales_all_features_cl_2', "wb"))
pickle.dump(scal, open('scales_all_features', "wb"))

# pickle.dump(xgb_models_cl[y_label], open('all_features_and_scales_all_features_cl', "wb"))

In [ ]:
def get_importance(model):
    importance = model.get_fscore()
    tuples = [(k, importance[k]) for k in importance]
    tuples = sorted(tuples, reverse=True, key=lambda x: x[1])
    labels, values = zip(*tuples)
    return labels, values
y_label = 'Ret_PlusOne'
l,v = get_importance(xgb_models_cl_2[y_label] )

In [ ]:
# f = l[0:250]
# v

In [ ]:
y_label = 'Ret_PlusOne'
y_pred = xgb_models[y_label].predict(X_Dvalids[y_label])
y_value = X_Dvalids[y_label].get_label()
print y_pred, y_value

In [ ]:
y_pred_cl = xgb_models_cl[y_label].predict(X_Dvalids_CL[y_label])
y_value_cl = X_Dvalids_CL[y_label].get_label()
print y_pred_cl, y_value_cl
print np.argmax(y_pred_cl, axis=1)
cl = np.argmax(y_pred_cl, axis=1)
prob_cl = np.max(y_pred_cl, axis=1)

In [ ]:
y_pred_cl_2 = xgb_models_cl_2[y_label].predict(X_Dvalids_CL_2[y_label])
y_value_cl_2 = X_Dvalids_CL_2[y_label].get_label()
print y_pred_cl_2, y_value_cl_2
print np.argmax(y_pred_cl_2, axis=1)
cl_2 = np.argmax(y_pred_cl_2, axis=1)

prob_cl_2 = np.max(y_pred_cl_2, axis=1)

In [ ]:
y = cl.copy()
y = y.astype(float64)
y[(cl_2 == 0)] = -0.0002582153
y[(cl_2 == 1)] = 0.0002
print wmae(-0.000258483738816342, X_Dvalids_CL_2[y_label])
print wmae(y, X_Dvalids_CL_2[y_label])

In [ ]:
print wmae(0, X_Dvalids[y_label])
print wmae(-0.000258483738816342, X_Dvalids[y_label])
print wmae(y_pred, X_Dvalids[y_label])
y_pred_copy = y_pred.copy()

y_pred_copy[y_pred == y_pred] = 0
condition_vec = (  ((cl_2 == 0) & (y_pred < 0)) | 
            ((cl_2 == 1) & (y_pred > 0)) )


# condition_vec = (  (((cl_2 == 0) & (y_pred < 0)) | (y_cl == cl)) | 
#             (((cl_2 == 1) & (y_pred > 0)) | (y_cl == cl)) )
y_pred_copy[condition_vec] = y_pred[condition_vec]

# y_pred_copy[((cl == 0) & (y_pred > 0) & (cl_2 == 0)) | 
#             ((cl == 2) & (y_pred < 0) & (cl_2 == 1)) | 
#             ((cl == 1))] = 0
# y_pred_copy[((cl == 0) & (y_pred > 0)) | 
#             ((cl == 2) & (y_pred < 0)) | 
#             ((cl == 1))] = 0
print wmae(y_pred_copy, X_Dvalids[y_label])

# ('wmae', 27709.582)
# ('wmae', 27700.25)
# ('wmae', 25733.922)
# ('wmae', 25857.531)


# most 800 features
# ('wmae', 27709.582)
# ('wmae', 27700.25)
# ('wmae', 26655.352)
# ('wmae', 26371.129)


In [ ]:
y_cl = pd.cut(y_pred,(-1, median_neg[y_label] * .7 , median_pos[y_label] * .7, 1),
        labels=(0, 1, 2))

In [ ]:
mean(y_cl == cl)

In [ ]:
# X_trains['Ret_PlusOne'].Feature_7
sum((cl == 2) & (y_pred_copy > 0))
print sum(((y_pred_copy > 0) & (y_value > 0))) + sum(((y_pred_copy < 0) & (y_value < 0)))
print sum(((y_pred_copy < 0) & (y_value > 0))) + sum(((y_pred_copy > 0) & (y_value < 0)))

print 2479. / (1414 + 2479) 
print sum(((y_pred > 0) & (y_value > 0))) + sum(((y_pred < 0) & (y_value < 0)))
print sum(((y_pred < 0) & (y_value > 0))) + sum(((y_pred > 0) & (y_value < 0)))
print 3518. / (3518 + 2482)

In [ ]:
import matplotlib.pyplot as plt
t = range(0, y_pred.shape[0])
plt.figure(figsize=(70, 30))
plt.scatter(t,
            y_pred_copy,
            s= 500*prob_cl_2,
            c='green')

plt.scatter(t,
            y_value,
            s= 100,
            c='red')

plt.axhline(0, color='black')
plt.xlim(500, 3900)
# plt.axhline(median_neg[y_label], color='black')
# plt.axhline(median_pos[y_label], color='black')
# plt.axhline(-0.021499999999917918, color='black')
# plt.axhline(0.0254999999999, color='black')
# print(mean(y_prob[:, 0] > .9))

In [ ]:
xgb_models = dict()
xgb_models_cl = dict()
xgb_models_cl_2 = dict()
y_label = 'Ret_PlusOne'
# xgb_models[y_label] = dc['reg']
# xgb_models_cl[y_label] = dc['cl']
# xgb_models_cl_2[y_label] = dc['cl_2']

xgb_models[y_label] = pickle.load(open('all_features_and_scales_all_featuresreg', "rb"))
xgb_models_cl_2[y_label] = pickle.load(open('all_features_and_scales_all_features_cl_2', "rb"))
scal = pickle.load(open('scales_all_features', "rb"))

In [ ]:
# test_df = pd.read_csv('./test.data.new.sum.csv')
test_df = pd.read_csv('./test.data.new_prod.csv')
y_label = 'Ret_PlusOne'

In [ ]:
# X_tests[y_label].Feature_7.head()

test_c7 = pd.read_csv('./c7.test.csv')
test_df.Feature_7 = test_c7.c7_50
test_df['c7'] = test_c7.c7
test_df.Feature_7.head()

In [ ]:
max_ind = np.argmax(test_df.ix[:, -120:-1].as_matrix(), axis=1)
min_ind = np.argmin(test_df.ix[:, -120:-1].as_matrix(), axis=1)

test_df = pd.concat([pd.DataFrame(min_ind, columns=['min_ret']),
               pd.DataFrame(max_ind, columns=['max_ret']),
               test_df], axis=1)
 
feature_list = test_df.filter(regex=r'cumprod', axis=1).columns

max_ind = np.argmax(test_df.ix[:, feature_list].as_matrix(), axis=1)
min_ind = np.argmin(test_df.ix[:, feature_list].as_matrix(), axis=1)

test_df = pd.concat([pd.DataFrame(min_ind, columns=['min_cum']),
               pd.DataFrame(max_ind, columns=['max_cum']), test_df], axis=1)

feature_list = test_df.filter(regex=r'cumprod', axis=1).columns

std =    np.std(test_df.ix[:, -120:-1].as_matrix(), axis=1)
median = np.median(test_df.ix[:, -120:-1].as_matrix(), axis=1)

test_df = pd.concat([pd.DataFrame(std, columns=['std']),
               pd.DataFrame(median, columns=['median']), test_df], axis=1)

test_df.drop('Id', axis=1, inplace=True)

In [ ]:
%xdel median
%xdel std
%xdel min_ind
%xdel max_ind

import gc
gc.collect()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler


In [ ]:
# X_sub.head()
# test_df.drop('Id', axis=1, inplace=True)
# len(new_columns)
# scal.transform(X_sub.ix[:, 1766:3520])

In [ ]:
feature_list = xgb_models[y_label].feature_names
X_sub = test_df.ix[0:60000, feature_list[1760:3520]]
# new_columns  = test_df.columns
new_columns = [x + '_Scaler' for x in feature_list[1760:3520].tolist()]
X_sub = pd.concat([pd.DataFrame(scal.transform(X_sub), columns=new_columns), 
              X_sub], axis=1)

X_Dsub = xgb.DMatrix(X_sub.ix[:, feature_list])
y_pred_0 = xgb_models[y_label].predict(X_Dsub)
pickle.dump(y_pred_0, open('y_pred_0', 'wb'))

In [ ]:
X_sub.head()

In [ ]:
%xdel X_Dsub
%xdel X_sub
import gc
gc.collect()

In [ ]:
X_sub.head()

In [ ]:
feature_list = xgb_models[y_label].feature_names
X_sub = test_df.ix[60001:120000, feature_list[1760:3520]]
# new_columns  = test_df.columns
new_columns = [x + '_Scaler' for x in feature_list[1760:3520]]
X_sub = pd.concat([pd.DataFrame(scal.transform(X_sub), columns=new_columns, index=X_sub.index), 
              X_sub], axis=1)

import gc
gc.collect()

X_Dsub = xgb.DMatrix(X_sub.ix[:, feature_list])
y_pred_1 = xgb_models[y_label].predict(X_Dsub)
pickle.dump(y_pred_1, open('y_pred_1', 'wb'))

In [ ]:
feature_list = xgb_models_cl_2[y_label].feature_names
X_sub = test_df.ix[0:60000, feature_list[1760:3520]]
# new_columns  = test_df.columns
new_columns = [x + '_Scaler' for x in feature_list[1760:3520]]
X_sub = pd.concat([pd.DataFrame(scal.transform(X_sub), columns=new_columns), 
              X_sub], axis=1)

X_Dsub = xgb.DMatrix(X_sub.ix[:, feature_list])
y_pred_0_CL_2 = xgb_models_cl_2[y_label].predict(X_Dsub)

%xdel X_Dsub
%xdel X_sub
import gc
gc.collect()

X_sub = test_df.ix[60001:120000, feature_list[1760:3520]]
# new_columns  = test_df.columns
X_sub = pd.concat([pd.DataFrame(scal.transform(X_sub), columns=new_columns, index=X_sub.index), 
              X_sub], axis=1)

X_Dsub = xgb.DMatrix(X_sub.ix[:, feature_list])
y_pred_1_CL_2 = xgb_models_cl_2[y_label].predict(X_Dsub)

%xdel X_Dsub
%xdel X_sub
import gc
gc.collect()

In [ ]:
pickle.dump(y_pred_0_CL_2, open('y_pred_0_CL_2', 'wb'))
pickle.dump(y_pred_1_CL_2, open('y_pred_1_CL_2', 'wb'))

In [ ]:
# feature_list = xgb_models[y_label].feature_names
# X_sub = test_df.ix[60001:120000, feature_list]
# X_Dsub = xgb.DMatrix(X_sub)
# y_pred_1 = xgb_models[y_label].predict(X_Dsub)

In [ ]:
%xdel X_Dsub
%xdel X_sub
import gc
gc.collect()

In [ ]:
feature_list = xgb_models_cl[y_label].feature_names
X_sub = test_df.ix[0:60000, feature_list]
X_Dsub = xgb.DMatrix(X_sub)
y_pred_0_CL = xgb_models_cl[y_label].predict(X_Dsub)
%xdel X_Dsub
%xdel X_sub
gc.collect()
X_sub = test_df.ix[60001:120000, feature_list]
X_Dsub = xgb.DMatrix(X_sub)
y_pred_1_CL = xgb_models_cl[y_label].predict(X_Dsub)

In [ ]:
feature_list = xgb_models_cl_2[y_label].feature_names
X_sub = test_df.ix[0:60000, feature_list]
X_Dsub = xgb.DMatrix(X_sub)
y_pred_0_CL_2 = xgb_models_cl_2[y_label].predict(X_Dsub)
%xdel X_Dsub
%xdel X_sub
gc.collect()
X_sub = test_df.ix[60001:120000, feature_list]
X_Dsub = xgb.DMatrix(X_sub)
y_pred_1_CL_2 = xgb_models_cl_2[y_label].predict(X_Dsub)

In [ ]:
y_pred_0_CL_2 = pickle.load(open('y_pred_0_CL_2', 'rb'))
y_pred_1_CL_2 = pickle.load(open('y_pred_1_CL_2', 'rb'))

In [ ]:
y_pred_0 = pickle.load(open('y_pred_0', 'rb'))
y_pred_1 = pickle.load(open('y_pred_1', 'rb'))

In [ ]:
y_pred = np.concatenate((y_pred_0, y_pred_1), axis = 0)

In [ ]:
y_pred_copy = cl_2.copy()
y_pred_copy = y_pred_copy.astype(float64)
y_pred_copy[cl_2 == 0] = -0.000258483738816342
y_pred_copy[cl_2 == 1] = 0.0002

In [ ]:
cl_2 == 0
y_pred_copy

In [ ]:
# y_pred_CL = np.concatenate((y_pred_0_CL, y_pred_1_CL), axis = 0)
y_pred_CL_2 = np.concatenate((y_pred_0_CL_2, y_pred_1_CL_2), axis = 0)

In [ ]:
# y_pred[abs(y_pred) > 0.2] = 0
# sum(abs(y_pred) > 0.2)



# cl = np.argmax(y_pred_CL, axis=1)
cl_2 = np.argmax(y_pred_CL_2, axis=1)
# prob_cl = np.max(y_pred_CL, axis=1)
prob_cl_2 = np.max(y_pred_CL_2, axis=1)

In [ ]:
y_pred_copy = y_pred.copy()
y_pred_copy[y_pred == y_pred] = 0
print sum(y_pred_copy)
condition_vec = (  ((cl_2 == 0) & (y_pred < 0)) | 
            ((cl_2 == 1) & (y_pred > 0)) )
y_pred_copy[condition_vec] = y_pred[condition_vec]

In [ ]:
sum(y_pred_copy != 0)


In [ ]:
y_pred_copy.shape
print sum(  ((cl == 0) & (y_pred > 0)) | 
            ((cl == 2) & (y_pred < 0)) | 
            ((cl == 1)) ) # <- 0

In [ ]:
print sum(abs(y_pred_copy) > 0.1)
y_pred_copy[abs(y_pred_copy) > 0.1] = 0
print sum(abs(y_pred_copy) > 0.1)

In [ ]:
y_pred_copy[1:5]

In [ ]:
subm_df = pd.read_csv('./sample_submission_2.csv', header=0, index_col=0)
s = (test_df.Id.astype(int64).astype(str) + '_61')
subm_df.loc[s.as_matrix(), 'Predicted'] = y_pred_copy / 2
# sum(subm_df.Predicted < 0)
subm_df.to_csv('./cl2_all.features.standard.Scaler_median_div2.csv')

In [ ]:
y_label + suffix_file

In [ ]:
xgb_models

In [ ]:
# dict_to_save = {
#     'reg':xgb_models[y_label],
#     'cl':xgb_models_cl[y_label],
#     'cl_2':xgb_models_cl_2[y_label]
# }
# pickle.dump(dict_to_save, open(y_label + '_all_features', "wb"))
pickle.dump(xgb_models[y_label], open('all_features_and_scales_all_features_reg', "wb"))
# pickle.dump(xgb_models_cl_2[y_label], open('all_features_and_scales_all_features_cl_2', "wb"))
# pickle.dump(xgb_models_cl[y_label], open('all_features_and_scales_all_features_cl', "wb"))

In [ ]:
# dc = pickle.load(open('Ret_PlusOne_clf_Feature__OSMA_MOM_RSI_ALMA_Ret_.pkl', "rb"))
# dc = pickle.load(open('Ret_PlusOne_clf_Feature__MOM_RSI_Ret_.pkl', "rb"))
# dc = pickle.load(open('Ret_PlusOne_clf_Feature__MOM_ALMA_OSMA_MACD_RSI_f7_c100_cumsum.pkl', "rb"))
dc = pickle.load(open('Ret_PlusOne_all_features', "rb"))
dc

In [ ]:
xgb_models = dict()
xgb_models_cl = dict()
xgb_models_cl_2 = dict()
y_label = 'Ret_PlusOne'
xgb_models[y_label] = dc['reg']
xgb_models_cl[y_label] = dc['cl']
xgb_models_cl_2[y_label] = dc['cl_2']

In [ ]:
1000*prob_cl_2

In [ ]:
import matplotlib.pyplot as plt
t = range(0, y_pred.shape[0])
plt.figure(figsize=(70, 30))
plt.scatter(t,
            y_pred_copy/10,
            s= 1000*prob_cl_2,
            c='green')
plt.yticks(fontsize=36)

plt.axhline(0, color='black')
# plt.axhline(0.01, color='black')
# plt.axhline(-0.010, color='black')
plt.xlim(1000, 1900)
plt.ylim(-0.001, 0.001)

In [ ]:
y_cl_2 = np.max(y_pred_CL_2, axis=1)
print y_cl_2, y_pred_CL_2


# Neural network

In [ ]:
from sklearn.svm import SVR
from sknn.mlp import Regressor, Layer
from sklearn.grid_search import GridSearchCV

In [ ]:
nn = Regressor(
    layers=[
        Layer("Rectifier", units=250),
        Layer("Sigmoid", units=50),
        Layer("Tanh", units=20),
        Layer("Sigmoid", units=10),
        Layer("Linear")],
    learning_rate=0.002,
    n_iter=10)

X_train = X_trains['Ret_PlusOne'].ix[:, :-1]

# gs = GridSearchCV(nn, param_grid={
#     'learning_rate': [0.002],
#     'hidden0__units': [400],
#     'hidden0__type': ["Rectifier", "Tanh", "ExpLin"],
#     'hidden1__units': [800, 1200],
#     'hidden1__type': ["Sigmoid", "Tanh"],
#     'output__type': ["Sigmoid", "Linear"]})


In [ ]:
nn.fit(X_train.as_matrix(),
       y_trains['Ret_PlusOne'].as_matrix())

In [ ]:
# X_trains['Ret_PlusOne'].head() #.ix[:, :-1]
# np.logspace(-2, 2, 5)

In [ ]:
y = nn.predict(X_tests['Ret_PlusOne'].ix[:, :-1].as_matrix())
y = y[:,0]

In [ ]:
# y

In [ ]:

y[y == y[0]] = 0

In [ ]:
mean(X_tests['Ret_PlusOne'].ix[:, -1])

In [ ]:
def get_wmae(preds, target, weight):
    x1 = np.abs(target - preds)
    x2 = x1 * weight
    x3 = np.mean(x2)
    return x3

In [ ]:
print get_wmae(y, y_tests['Ret_PlusOne'], X_tests['Ret_PlusOne'].ix[:, -1])
print get_wmae(0, y_tests['Ret_PlusOne'], X_tests['Ret_PlusOne'].ix[:, -1])

In [ ]:
y_tar = y_tests['Ret_PlusOne']
# y_tar[y_tar < 0] = -0.0005
# y_tar[y_tar > 0] = 0.0005

In [ ]:
import matplotlib.pyplot as plt
t = range(0, y.shape[0])
plt.figure(figsize=(70, 30))
plt.scatter(t,
            y,
            s = 150,
            c='green')
plt.scatter(t,
            y_tar,
            s = 150,
            c='red')
plt.yticks(fontsize=36)

plt.axhline(0, color='black')
# plt.axhline(0.01, color='black')
# plt.axhline(-0.010, color='black')
plt.xlim(1000, 1300)
plt.ylim(-0.001, 0.001)

In [ ]:
# X.ix[:, -120:-1].head().as_matrix()